In [ ]:
import pandas as pd
import ollama
from pydantic import BaseModel
import json
import os
import time

pd.options.mode.chained_assignment = None

In [8]:
# Specify model
model = "mistral-small3.2"

In [9]:
class MacronutrientEstimations(BaseModel):
    simple_sugars: int
    complex_sugars: int
    proteins: int
    fats: int
    dietary_fibers: int
    explanation: str

macronutrients_instruction = f'''Examine the provided meal image to analyze and estimate its nutritional content accurately. 
Determine the amounts of simple sugars (like industrial sugar and honey), complex sugars (such as starch and whole grains), proteins, fats, and dietary fibers (found in fruits and vegetables), all in grams.
To assist in accurately gauging the scale of the meal, a 1 Swiss Franc coin, which has a diameter of 23 mm, may be present in the picture. 
Provide your assessment of each nutritional component in grams. All estimates should be given as a single whole number. 
Use the size of this coin as a reference to estimate the size of the meal and the amounts of the nutrients more precisely. 
If there is no coin in the picture or the meal is covered partially, estimate anyways.


Here are three examples of how to analyze different types of foods:

Example 1 - Pizza:
{MacronutrientEstimations(
    simple_sugars=6,
    complex_sugars=65,
    proteins=28,
    fats=32,
    dietary_fibers=5,
    explanation="This appears to be a medium-sized cheese pizza (about 12 inches in diameter). The crust provides most of the complex sugars (approximately 65g from flour). The tomato sauce contains some simple sugars (6g). The cheese provides most of the protein (28g) and fats (32g). The small amount of tomato sauce and potentially some vegetables contribute to the dietary fiber content (5g)."
)}

Example 2 - Bread with cheese and salad:
{MacronutrientEstimations(
    simple_sugars=4,
    complex_sugars=30,
    proteins=18,
    fats=15,
    dietary_fibers=8,
    explanation="The meal consists of a slice of bread with cheese and a side salad. The bread provides most of the complex sugars (30g from wheat flour). The cheese contributes protein (10g) and fats (10g). The salad provides dietary fibers (8g) and a small amount of simple sugars (4g from vegetables). The bread also contributes some protein (8g) and fats (5g)."
)}

Example 3 - Bottle of orange juice:
{MacronutrientEstimations(
    simple_sugars=42,
    complex_sugars=2,
    proteins=2,
    fats=0,
    dietary_fibers=1,
    explanation="This appears to be a standard 500ml bottle of orange juice. Orange juice is primarily composed of simple sugars (42g from natural fruit sugars). It contains minimal complex sugars (2g), a small amount of protein (2g), negligible fat (0g), and a small amount of dietary fiber (1g) from pulp residue in the juice."
)}

Format your response in the following JSON format:
{MacronutrientEstimations(
    simple_sugars=0,
    complex_sugars=0,
    proteins=0,
    fats=0,
    dietary_fibers=0,
    explanation=''
)}'''

In [ ]:
def ollama_food_annotations(image_path):

    res = ollama.chat(
        model=model,
        format=MacronutrientEstimations.model_json_schema(),
        messages=[
            {
                'role': 'user',
                'content': macronutrients_instruction,
                'images': [image_path]
            }
        ]
    )

    return res['message']['content']

In [ ]:
patients = ['001', '002', '004', '006', '007', '008']
features = ['simple_sugars', 'complex_sugars', 'fats', 'dietary_fibers', 'proteins', 'fast_insulin', 'slow_insulin']

for patient in patients: 
    print(f"Processing patient {patient}")
    food_data_path = f"../diabetes_subset_pictures-glucose-food-insulin/{patient}/food.csv"
    food_data = pd.read_csv(food_data_path)
    food_data['simple_sugars'] = 0.0
    food_data['complex_sugars'] = 0.0 
    food_data['proteins'] = 0.0
    food_data['fats'] = 0.0
    food_data['dietary_fibers'] = 0.0
    food_data['message'] = ''
    
    for i, row in food_data.iterrows():
        image_path = f"../diabetes_subset_pictures-glucose-food-insulin/{patient}/food_pictures/{row['picture']}"
        max_retries = 3
        retry_count = 0
        while retry_count < max_retries:
            try:
                message = ollama_food_annotations(image_path)
                estimated_macros = MacronutrientEstimations.model_validate_json(message)
                food_data.loc[i, 'simple_sugars'] = estimated_macros.simple_sugars
                food_data.loc[i, 'complex_sugars'] = estimated_macros.complex_sugars
                food_data.loc[i, 'proteins'] = estimated_macros.proteins
                food_data.loc[i, 'fats'] = estimated_macros.fats
                food_data.loc[i, 'dietary_fibers'] = estimated_macros.dietary_fibers
                food_data.loc[i, 'message'] = estimated_macros.explanation
                break
            except (json.JSONDecodeError, KeyError) as e:
                print(f"Error processing image {i}, attempt {retry_count + 1}: {str(e)}")
                retry_count += 1
                if retry_count == max_retries:
                    print(f"Failed to process image {i} after {max_retries} attempts")
                time.sleep(5)  
    if not os.path.exists(f'../food_data/{model}'):
        os.makedirs(f'../food_data/{model}')
    food_data.to_csv(f'../food_data/{model}/{patient}.csv', index=False)

Processing patient 001
Processing patient 002
Processing patient 004
Processing patient 006
Processing patient 007
Processing patient 008
